In [26]:
import requests
import pandas as pd


In [27]:


def get_lost_objects(gares, annees):
    results = []
    for gare in gares:
        for annee in annees:
            response = requests.get(f'https://a67b4b4d-cb00-45aa-9970-2c69b46cea8a@ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&lang=fr&rows=10000&sort=date&facet=date&facet=gc_obo_gare_origine_r_name&facet=gc_obo_type_c&refine.date={annee}&refine.gc_obo_gare_origine_r_name={gare}')
            results.append(response.json()["records"])
    return results



In [28]:
db_gares = pd.read_csv("ressources/gares.csv")

gares = [name.replace("'","%27") for name in db_gares.Gare]
annees = [annee for annee in range(2019,2020)]

lost_objects_list = get_lost_objects(gares, annees)

In [29]:
gares

['Paris Gare du Nord',
 'Paris Est',
 'Paris Gare de Lyon',
 'Paris Montparnasse',
 'Paris Saint-Lazare',
 'Paris Austerlitz',
 'Paris Bercy']

In [34]:
lost_objects_list[0][]

{'datasetid': 'objets-trouves-restitution',
 'recordid': '9d14fba406ba8c26aa6346bc8265803a29269eac',
 'fields': {'gc_obo_gare_origine_r_code_uic_c': '0087271007',
  'gc_obo_type_c': 'Vêtements, chaussures',
  'gc_obo_gare_origine_r_name': 'Paris Gare du Nord',
  'gc_obo_nature_c': 'Manteau, veste, blazer, parka, blouson, cape',
  'gc_obo_nom_recordtype_sc_c': 'Objet trouvé',
  'date': '2019-10-15T16:46:12+00:00'},
 'record_timestamp': '2023-04-06T13:00:52.831Z'}